In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count, chain
import tqdm
import copy

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.autograd import Variable


from diffusers import DDIMScheduler, DPMSolverMultistepScheduler, EulerAncestralDiscreteScheduler

from nosaveddata.nsd_utils.save_hypers import Hypers, nsd_Module
from nosaveddata.nsd_utils.nsd_csv import add_to_csv
from nosaveddata.nsd_utils.networks import params_count, params_and_grad_norm, seed_np_torch
from nosaveddata.nsd_utils.einstein import Rearrange

from nosaveddata.builders.mlp import *
from nosaveddata.builders.weight_init import *
from nosaveddata.builders.resnet import IMPALA_Resnet, DQN_Conv, IMPALA_YY

from nosaveddata import *


from utils.experience_replay import *


import locale
locale.getpreferredencoding = lambda: "UTF-8"

import wandb


# Environment configuration
#env_name = 'Kangaroo'
#SEED = 8712

env_name = 'Assault'
SEED = 7783


wandb.init(
    project="Atari-100k-BBF",
    name=f"BBF-Diffusion-{env_name}",

    #id='rotdmtc5',
    #resume='must',

    config={
        "learning_rate": 1e-4,
        "architecture": "BBF",
        "dataset": "Assault",
        "epochs": 100,
    },

    reinit=False
)



# Optimization
batch_size = 32
lr=1e-4
lr_diffusion=1e-4
lr_energy=1e-4
M=16
energy_beta=3
HIDDENS=2048

eps=1e-8


# Target network EMA rate
critic_ema_decay=0.995


# Return function
initial_gamma=torch.tensor(1-0.97).log()
final_gamma=torch.tensor(1-0.997).log()

initial_n = 10
final_n = 3

num_buckets=51


# Reset Schedule and Buffer
reset_every=40000 # grad steps, not steps.
schedule_max_step=reset_every//4
total_steps=102000

prefetch_cap=1 # actually, no prefetch is being done


Transition = namedtuple('Transition',
                        ('state', 'reward', 'action', 'c_flag'))
memory = PrioritizedReplay_nSteps_Sqrt(total_steps+5, total_steps=schedule_max_step, prefetch_cap=prefetch_cap)



def save_checkpoint(net, model_target, optimizer, step, path):
    torch.save({
            'model_state_dict': net.state_dict(),
            'model_target_state_dict': model_target.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'step': step,
            }, path)


C:\Users\sneep\anaconda3\envs\python_\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\sneep\anaconda3\envs\python_\Lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
wandb: Currently logged in as: snykralafk. Use `wandb login --relogin` to force relogin


In [2]:
# Adapted from: https://github.com/weipu-zhang/STORM/blob/main/env_wrapper.py
class MaxLast2FrameSkipWrapper(Hypers, gym.Wrapper):
    def __init__(self, env, skip=4, noops=30, seed=0):
        super().__init__(env=env)
        self.env.action_space.seed(seed)
        
    def reset(self, **kwargs):
        kwargs["seed"] = self.seed
        obs, _ = self.env.reset(**kwargs)

        return obs, _
        
    def noop_steps(self, states):
        noops = random.randint(0,self.noops)
        
        for i in range(noops):
            state = self.step(np.array([0]))[0]
            state = preprocess(state)
            states.append(state)
        return states

    def step(self, action):
        total_reward = 0
        self.obs_buffer = deque(maxlen=2)
        for _ in range(self.skip):
            obs, reward, done, truncated, info = self.env.step(action)
            self.obs_buffer.append(obs)
            total_reward += reward

            terminated = np.logical_or(done, truncated)
            #if terminated.any():
            #    for i in range(len(terminated)):
            #       obs[i] = self.reset()[0][i]
            if done or truncated:
                break
        if len(self.obs_buffer) == 1:
            obs = self.obs_buffer[0]
        else:
            obs = np.max(np.stack(self.obs_buffer), axis=0)
        return obs, total_reward, done, truncated, info
        # Life loss is calculated on the training code

env = gym.vector.make(f"{env_name}NoFrameskip-v4", num_envs=1)
env = MaxLast2FrameSkipWrapper(env,seed=SEED)


#n_actions = env.action_space.n
n_actions = env.action_space[0].n

state, info = env.reset()
n_observations = len(state)

seed_np_torch(SEED)

C:\Users\sneep\anaconda3\envs\python_\Lib\site-packages\gymnasium\vector\__init__.py:53: UserWarning: WARN: `gymnasium.vector.make(...)` is deprecated and will be replaced by `gymnasium.make_vec(...)` in v1.0
  gym.logger.warn(


In [3]:

def renormalize(tensor, has_batch=False):
    shape = tensor.shape
    tensor = tensor.view(tensor.shape[0], -1)
    max_value,_ = torch.max(tensor, -1, keepdim=True)
    min_value,_ = torch.min(tensor, -1, keepdim=True)
    return ((tensor - min_value) / (max_value - min_value + 1e-5)).view(shape)


class DQN(nn.Module):
    def __init__(self, n_actions, hiddens=HIDDENS, mlp_layers=1, scale_width=4,
                 n_atoms=51, Vmin=-10, Vmax=10):
        super().__init__()
        self.support = torch.linspace(Vmin, Vmax, n_atoms).cuda()
        
        self.hiddens=hiddens
        self.scale_width=scale_width
        self.act = nn.ReLU()
        
        
        self.encoder_cnn = IMPALA_Resnet(scale_width=scale_width, norm=False, init=init_xavier, act=self.act)
        
        

        # Single layer dense that maps the flattened encoded representation into hiddens.
        self.projection = MLP(13824, med_hiddens=hiddens, out_hiddens=hiddens,
                              last_init=init_xavier, layers=1)
        self.prediction = MLP(hiddens, out_hiddens=hiddens, layers=1, last_init=init_xavier)
                                              
        self.transition = nn.Sequential(DQN_Conv(32*scale_width+n_actions, 32*scale_width, 3, 1, 1, norm=False, init=init_xavier, act=self.act),
                                        DQN_Conv(32*scale_width, 32*scale_width, 3, 1, 1, norm=False, init=init_xavier, act=self.act))

        # Single layer dense that maps hiddens into the output dim according to:
        # 1. https://arxiv.org/pdf/1707.06887.pdf -- Distributional Reinforcement Learning
        # 2. https://arxiv.org/pdf/1511.06581.pdf -- Dueling DQN
        self.a = MLP(hiddens, out_hiddens=n_actions*num_buckets, layers=1, in_act=self.act, last_init=init_xavier)
        self.v = MLP(hiddens, out_hiddens=num_buckets, layers=1, in_act=self.act, last_init=init_xavier)
    
        params_count(self, 'DQN')
    
    def forward(self, X, y_action):
        X, z = self.encode(X)
        
        
        q, action = self.q_head(X)
        z_pred = self.get_transition(z, y_action)

        return q, action, X[:,1:].clone().detach(), z_pred
    

    def env_step(self, X):
        with torch.no_grad():
            X, _ = self.encode(X)
            _, action = self.q_head(X)
            
            return action.detach()
    

    def encode(self, X):
        batch, seq = X.shape[:2]
        self.batch = batch
        self.seq = seq
        X = self.encoder_cnn(X.contiguous().view(self.batch*self.seq, *(X.shape[2:])))
        X = renormalize(X).contiguous().view(self.batch, self.seq, *X.shape[-3:])
        X = X.contiguous().view(self.batch, self.seq, *X.shape[-3:])
        z = X.clone()
        X = X.flatten(-3,-1)
        X = self.projection(X)
        return X, z

    def get_transition(self, z, action):
        z = z.contiguous().view(-1, *z.shape[-3:])
        
        action = F.one_hot(action.clone(), n_actions).view(-1, n_actions)
        action = action.view(-1, 5, n_actions, 1, 1).expand(-1, 5, n_actions, *z.shape[-2:])

        z_pred = torch.cat( (z, action[:,0]), 1)
        z_pred = self.transition(z_pred)
        z_pred = renormalize(z_pred)
        
        z_preds=[z_pred.clone()]
        

        for k in range(4):
            z_pred = torch.cat( (z_pred, action[:,k+1]), 1)
            z_pred = self.transition(z_pred)
            z_pred = renormalize(z_pred)
            
            z_preds.append(z_pred)
        
        
        z_pred = torch.stack(z_preds,1)

        z_pred = self.projection(z_pred.flatten(-3,-1)).view(self.batch,5,-1)
        z_pred = self.prediction(z_pred)
        
        return z_pred

    
    def q_head(self, X):
        q = self.dueling_dqn(X)
        action = (q*self.support).sum(-1).argmax(-1)
        
        return q, action

    def get_max_action(self, X):
        with torch.no_grad():
            X, _ = self.encode(X)
            q = self.dueling_dqn(X)
            
            action = (q*self.support).sum(-1).argmax(-1)
            return action

    def evaluate(self, X, action):
        with torch.no_grad():
            X, _ = self.encode(X)
            
            q = self.dueling_dqn(X)
            
            action = action[:,:,None,None].expand_as(q)[:,:,0][:,:,None]
            q = q.gather(-2,action)
            
            return q

    def dueling_dqn(self, X):
        X = F.relu(X)
        
        a = self.a(X).view(self.batch, -1, n_actions, num_buckets)
        v = self.v(X).view(self.batch, -1, 1, num_buckets)
        
        q = v + a - a.mean(-2,keepdim=True)
        q = F.softmax(q,-1)
        
        return q
    
    def network_ema(self, rand_network, target_network, alpha=0.5):
        for param, param_target in zip(rand_network.parameters(), target_network.parameters()):
            param_target.data = alpha * param_target.data + (1 - alpha) * param.data.clone()

    def hard_reset(self, random_model, alpha=0.5):
        with torch.no_grad():
            
            self.network_ema(random_model.encoder_cnn, self.encoder_cnn, alpha)
            self.network_ema(random_model.transition, self.transition, alpha)

            self.network_ema(random_model.projection, self.projection, 0)
            self.network_ema(random_model.prediction, self.prediction, 0)

            self.network_ema(random_model.a, self.a, 0)
            self.network_ema(random_model.v, self.v, 0)



def copy_states(source, target):
    for key, _ in zip(source.state_dict()['state'].keys(), target.state_dict()['state'].keys()):

        target.state_dict()['state'][key]['exp_avg_sq'] = copy.deepcopy(source.state_dict()['state'][key]['exp_avg_sq'])
        target.state_dict()['state'][key]['exp_avg'] = copy.deepcopy(source.state_dict()['state'][key]['exp_avg'])
        target.state_dict()['state'][key]['step'] = copy.deepcopy(source.state_dict()['state'][key]['step'])
        
def target_model_ema(model, model_target):
    with torch.no_grad():
        for param, param_target in zip(model.parameters(), model_target.parameters()):
            param_target.data = critic_ema_decay * param_target.data + (1.0 - critic_ema_decay) * param.data.clone()


class EnergyModel(nsd_Module):
    def __init__(self, hiddens, n_actions, condition_dim, t_dim=32, Vmin=-10, Vmax=10, n_atoms=51, act=nn.ReLU()):
        super().__init__()
        self.support = torch.linspace(Vmin, Vmax, n_atoms).cuda()
        self.t_emb = MLP(t_dim, out_hiddens=t_dim, last_init=init_xavier, out_act=nn.Identity())
        
        self.a = MLP(hiddens+n_actions+t_dim+condition_dim, out_hiddens=n_actions*num_buckets, layers=1, in_act=self.act, last_init=init_xavier)
        self.v = MLP(hiddens+n_actions+t_dim+condition_dim, out_hiddens=num_buckets, layers=1, in_act=self.act, last_init=init_xavier)
        
        params_count(self, 'Energy Model')
        
    def forward(self, energy, t, condition):
        self.batch = 1 if energy.shape[0]==1 else batch_size
        t_emb = sinusoidal_embedding(t, self.t_dim)[:,None].repeat_interleave(energy.shape[-2], -2)
        t_emb = self.t_emb(t_emb)
        condition = condition[:,None].repeat_interleave(energy.shape[-2], -2)
        
        X = torch.cat((energy, t_emb, condition), -1)
        X = F.relu(X)
        
        a = self.a(X).view(self.batch, -1, n_actions, num_buckets)
        v = self.v(X).view(self.batch, -1, 1, num_buckets)
        
        
        q = v + a - a.mean(-2,keepdim=True)
        q = F.softmax(q,-1)
        
        action = (q*self.support).sum(-1).argmax(-1)
        
        return q, action

    def hard_reset(self, random_model, alpha=0):
        with torch.no_grad():
            
            network_ema(self.a, random_model.a, alpha)
            network_ema(self.v, random_model.v, alpha)
            network_ema(self.t_emb, random_model.t_emb, alpha)


class Diffusion(nsd_Module):
    def __init__(self, d_z, n_actions, d_model, num_blks=2, seq_len=5,
                 zero_cond_prob=0.25):
        super().__init__()
        
        self.unet = UNet_DiT_1D(d_z+n_actions, d_model, num_blks, d_model//64, seq_len=seq_len)

        self.s0_proj = MLP(d_z, out_hiddens=d_model, last_init=init_xavier, out_act=nn.SiLU())
        
    def forward(self, X, t, s0):
        
        zero_prob = torch.bernoulli(torch.ones(s0.shape[0])*(1-self.zero_cond_prob)).cuda()
        s0_emb = self.s0_proj(s0*zero_prob[:,None])
        
        return self.unet(X, t, s0_emb)

    def denoise(self, X, t, s0):
        s0_emb = self.s0_proj(s0)
        
        return self.unet(X, t, s0_emb)

    def hard_reset(self, random_model, alpha=0.5):
        with torch.no_grad():
            
            network_ema(self.unet, random_model.unet, alpha)
            network_ema(self.s0_proj, random_model.s0_proj, alpha)




model=DQN(n_actions).cuda()
model_target=DQN(n_actions).cuda()
diffusion_model = Diffusion(HIDDENS, n_actions, 512, num_blks=6).cuda()
energy_model = EnergyModel(HIDDENS, n_actions, HIDDENS).cuda()

model_target.load_state_dict(model.state_dict())



# Testing only
#with torch.no_grad():
#    q, action, X, z_pred = model(torch.randn(4,1,12,96,72, device='cuda', dtype=torch.float), torch.randint(0,n_actions,(4,5),device='cuda').long())
#z = model.encode(torch.randn(4,5,12,96,72, device='cuda'))[0]

# I believe the authors have actually miscalculated the params count on the paper.
# My training time is lower than theirs while having more parameters, and the same architecture is used as is their original code

IMPALA ResNet Parameters: 1.56M
DQN Parameters: 35.21M
IMPALA ResNet Parameters: 1.56M
DQN Parameters: 35.21M
UNet DIT Parameters: 233.96M
Energy Model Parameters: 1.69M


<All keys matched successfully>

In [4]:
perception_modules=[model.encoder_cnn, model.transition]
actor_modules=[model.prediction, model.projection, model.a, model.v]

params_wm=[]
for module in perception_modules:
    for param in module.parameters():
        if param.requires_grad==True: # They all require grad
            params_wm.append(param)

params_ac=[]
for module in actor_modules:
    for param in module.parameters():
        if param.requires_grad==True:
            params_ac.append(param)


optimizer = torch.optim.AdamW(chain(params_wm, params_ac),
                                lr=lr, weight_decay=0.1, eps=1.5e-4)
optimizer_diffusion = torch.optim.AdamW(diffusion_model.parameters(),
                                lr=lr_diffusion, weight_decay=0.)
optimizer_energy = torch.optim.AdamW(energy_model.parameters(),
                                lr=lr_energy, weight_decay=0.)

In [5]:
import torchvision.transforms as transforms

train_tfms = transforms.Compose([
                         transforms.Resize((96,72)),
                        ])


def preprocess(state):
    state=torch.tensor(state, dtype=torch.float, device='cuda') / 255
    state=train_tfms(state.permute(0,3,1,2))
    return state

# https://github.com/google/dopamine/blob/master/dopamine/jax/agents/dqn/dqn_agent.py
def linearly_decaying_epsilon(decay_period, step, warmup_steps, epsilon):
    steps_left = decay_period + warmup_steps - step
    bonus = (1.0 - epsilon) * steps_left / decay_period
    bonus = np.clip(bonus, 0., 1. - epsilon)
    return epsilon + bonus


def epsilon_greedy(states, step, grad_step, actions_to_step, final_eps=0, num_envs=1):

    if grad_step>=5500:
        s0, _ = model_target.encode(torch.cat(list(states),-3).unsqueeze(0))
        s0 = s0[:,0]
        
        latent = denoise(s0, std, samples=1)
        latent = latent.view(-1, *latent.shape[-2:])
        
        #energy, action = model_target.q_head(latent[...,:HIDDENS])
        #action = action.cpu()
        action = latent[...,-n_actions:].argmax(-1).cpu().squeeze()
        
        return list(action.to(torch.int64).split(1))
        
        
    
    epsilon = linearly_decaying_epsilon(2001, step, 2000, final_eps)

    if random.random() < epsilon:
        action = torch.randint(0, n_actions, (num_envs,), dtype=torch.int64).squeeze(0)
        actions_to_step.append(action)
    else:
        Q_action = model_target.env_step(torch.cat(list(states),-3).unsqueeze(0))
        action = Q_action.view(num_envs).squeeze(0).to(torch.int64).cpu()
        actions_to_step.append(action)
        
    return actions_to_step


In [6]:
#noise_scheduler = EulerAncestralDiscreteScheduler(num_train_timesteps=1000, beta_start=0.00085, beta_end=0.012)
noise_scheduler = DPMSolverMultistepScheduler(num_train_timesteps=1000, beta_start=0.00085, beta_end=0.012, use_karras_sigmas=False, solver_order=2)

def calculate_guidance(latent, t, s0, guidance_scale, samples, t_stopgrad=2):
    with torch.enable_grad():
        latent.requires_grad_(True)
        
        Q_t, a_t = energy_model(latent, torch.tensor([t]*latent.shape[0], device='cuda'), s0)
        a_t = a_t[...,None,None].expand_as(Q_t)[:,:,0][:,:,None]
        #print(f"{Q_t.shape, a_t.shape, latent.shape}")
        #print(f"{latent[...,-n_actions:]}")
        latent_action = latent[...,-n_actions:].argmax(-1).view(Q_t.shape[0],-1)[...,None,None].expand_as(Q_t)[:,:,0][:,:,None]
        Q_t = Q_t.gather(-2, latent_action)
        print(f"{(Q_t*model_target.support).sum(-1).sum()}")
        
        grad = guidance_scale * torch.autograd.grad(torch.sum(Q_t*model_target.support), latent)[0]
        
        latent.detach()
        
        grad[t<t_stopgrad] = 0
        return grad

        #return latent
    

def denoise(s0, std, inference_timesteps=30, guidance_scale=0.1, samples=1):
#def denoise(s0, inference_timesteps=100, guidance_scale=1.4, samples=1):
    # Thank you, Stable Diffusion
    #diffusion_model.eval()
    s0 = s0.repeat_interleave(samples, 0)

    z = torch.randn(s0.shape[0], 5, HIDDENS+n_actions, device='cuda') * noise_scheduler.init_noise_sigma
    
    
    # Run Model
    noise_scheduler.set_timesteps(inference_timesteps)
    
    
    for i, t in enumerate(noise_scheduler.timesteps):
        #z[:,0] = latent # from diffuser
        z_input = z

        z_input = noise_scheduler.scale_model_input(z_input, t)
        model_output = diffusion_model.denoise(z_input, torch.tensor([t]*z.shape[0], device='cuda'), s0)

        if guidance_scale!=0:
            
            #std_t = torch.sqrt(1.-torch.exp(2*torch.log(noise_scheduler.alphas_cumprod[999-t]))).view(-1,1,1)+eps
            std_t = torch.sqrt(1.-torch.exp(2*torch.log(noise_scheduler.alphas_cumprod[t.long()]))).view(-1,1,1)+eps
            std_t = std_t.cuda()

            sched_scale = (1-noise_scheduler.alphas_cumprod[t]).sqrt()
            model_output = model_output + sched_scale*calculate_guidance(model_output, t, s0, guidance_scale, samples)
            
            #model_output = model_output + calculate_guidance(model_output, t, s0, guidance_scale, samples)
            
            #model_output = -(model_output + calculate_guidance(model_output, t, s0, guidance_scale)*std_t)
            #model_output = model_output + calculate_guidance(model_output, t, s0, guidance_scale)*std_t
            
        
        
        #variance = noise_scheduler._get_variance(timestep, prev_timestep)
        #std_dev_t = eta * variance ** (0.5)

        # 2. compute previous image: x_t -> x_t-1
        z = noise_scheduler.step(
                model_output, t, z, return_dict=False# eta=1, use_clipped_model_output=False,
            )[0]
    #z[:,0] = latent
    noise_scheduler.set_timesteps(1000)
    #diffusion_model.train()
    print()
    return (z.view(-1,samples,*z.shape[1:]) if samples>1 else z)*std
    #return (z.view(-1,samples,*z.shape[1:]) if samples>1 else z)




def train_diffusion(latent, s0, same_traj, std, is_w, logs):
    
    diffusion_model.train()

    with torch.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=False):
    
        
        t = torch.randint(0,1000, (latent.shape[0],), device='cuda')
    
        noise = torch.randn_like(latent)
        t = noise_scheduler.timesteps.cuda()[t]
        perturbed_latent = noise_scheduler.add_noise(latent/std, noise, t)
        score = diffusion_model(perturbed_latent, t, s0)
        loss = mse(score, noise).sum(-2).view(batch_size,-1).mean(-1)
        #loss = (loss*is_w).mean()
        loss = loss.mean()
        

    loss.backward()
    #scaler_diffusion.scale(loss).backward()
    #scaler_diffusion.unscale_(optimizer_diffusion)
    torch.nn.utils.clip_grad_norm_(diffusion_model.parameters(), max_norm=10)
    #scaler_diffusion.step(optimizer_diffusion)
    #scaler_diffusion.update()
    optimizer_diffusion.step()
    optimizer_diffusion.zero_grad()
    
    logs['diffusion loss'] = loss.detach().cpu()
    return logs


def train_energy(latent, s0, same_traj, std, is_w, logs):
    
    #diffusion_model.eval()
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=False):
        with torch.no_grad():
            
            latent = denoise(s0, std=std, guidance_scale=0, samples=M)
            #latent = denoise(s0, guidance_scale=0, samples=M)
            #print(f"denoised {latent.shape}")
            latent = latent.view(-1, *latent.shape[-2:])
            #print(f"denoised view {latent.shape}")
            s0=s0.repeat_interleave(M,0)
            #print(f"s0 {s0.shape}")
            
            #energy, action = model_target.q_head(latent, alpha=3, seq=SEQ_LEN-1)
            energy, action = model_target.q_head(latent[...,:HIDDENS])
            latent_action = latent[...,-n_actions:].argmax(-1).view(batch_size,-1)[...,None,None].expand_as(energy)[:,:,0][:,:,None]
            #print(f"energy {energy.shape}")
            
            action = action[:,:,None,None].expand_as(energy)[:,:,0][:,:,None]
            #print(f"{action.shape, latent_action.shape}")
            energy = energy.gather(-2,latent_action)
            energy = (energy*model_target.support).sum(-1).view(batch_size,M,-1).sum(-1)
            
            #print(f"energy argmax {energy.shape}")
            
            energy = F.softmax(energy*energy_beta, -1)
            
            #print(f"energy softmax {energy.shape}")
    
        
        
        t = torch.randint(0,1000, (batch_size,), device='cuda')[:,None].repeat_interleave(M, 0)
        
        #print(f"latent pre {latent.shape}")
        latent = latent.view(batch_size*M, 5, -1)
        #print(f"latent view {latent.shape}")
        noise = torch.randn_like(latent)
        t = noise_scheduler.timesteps.cuda()[t]
        
        perturbed_latent = noise_scheduler.add_noise(latent/std, noise, t)
        
        energy_pred, action = energy_model(perturbed_latent, t.squeeze(), s0)#.squeeze()
        
        #print(f"energy pred: {energy_pred.shape, latent[...,-n_actions:].shape, action.shape}")
        latent_action = latent[...,-n_actions:].argmax(-1).view(batch_size,-1)[...,None,None].expand_as(energy_pred)[:,:,0][:,:,None]
        #print(f"{energy_pred.shape, latent_action.shape}")
        energy_pred = energy_pred.gather(-2,latent_action)#.view(batch_size, M, -1).sum(-1)
        energy_pred = (energy_pred*model_target.support).sum(-1).view(batch_size, M, -1).sum(-1)
        
        energy_pred = F.softmax(energy_pred, -1)
        
        loss = -(energy*torch.log(energy_pred+eps)).sum(-1) # (bz, M)
        loss = loss.mean()
        #loss = (loss*is_w).mean()
    
    loss.backward()
    #scaler_energy.scale(loss).backward()
    #scaler_energy.unscale_(optimizer_energy)
    #torch.nn.utils.clip_grad_norm_(energy_model.parameters(), max_norm=10)
    #scaler_energy.step(optimizer_energy)
    #scaler_energy.update()
    optimizer_energy.step()
    optimizer_energy.zero_grad()

    
    logs['energy'] = loss.detach().cpu()
    return logs   

In [7]:
# https://github.com/google/dopamine/blob/master/dopamine/jax/agents/rainbow/rainbow_agent.py
def project_distribution(supports, weights, target_support):
    with torch.no_grad():
        v_min, v_max = target_support[0], target_support[-1]
        # `N` in Eq7.
        num_dims = target_support.shape[-1]
        # delta_z = `\Delta z` in Eq7.
        delta_z = (v_max - v_min) / (num_buckets-1)
        # clipped_support = `[\hat{T}_{z_j}]^{V_max}_{V_min}` in Eq7.
        clipped_support = supports.clip(v_min, v_max)
        # numerator = `|clipped_support - z_i|` in Eq7.
        numerator = (clipped_support[:,None] - target_support[None,:,None].repeat_interleave(clipped_support.shape[0],0)).abs()
        quotient = 1 - (numerator / delta_z)
        # clipped_quotient = `[1 - numerator / (\Delta z)]_0^1` in Eq7.
        clipped_quotient = quotient.clip(0, 1)
        # inner_prod = `\sum_{j=0}^{N-1} clipped_quotient * p_j(x', \pi(x'))` in Eq7.
        inner_prod = (clipped_quotient * weights[:,None]).sum(-1)
        #inner_prod = (clipped_quotient).sum(-1) * weights
        return inner_prod.squeeze()


mse = torch.nn.MSELoss(reduction='none')
std = 1

scaler = torch.cuda.amp.GradScaler()
def optimize(step, grad_step, n):
    global std
    
    model.train()
    model_target.train()

    with torch.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=False):
        with torch.no_grad():
            states, next_states, rewards, action, c_flag, idxs, is_w = memory.sample(n, batch_size, grad_step)
            std = 0.99*std + 0.01*states.std(-1).mean()
            diffusion_actions = action
        terminal=1-c_flag
        #print(f"STUFF HERE {states.shape, rewards.shape, c_flag.shape, action.shape, n}")
    
    
        q, max_action, _, z_pred = model(states[:,0][:,None], action[:,:5].long())
        z = model_target.encode(states[:,:6])[0].detach()
        
        
        max_action  = model.get_max_action(next_states[:,n-1][:,None])
        next_values = model_target.evaluate(next_states[:,n-1][:,None].contiguous(), max_action)
        

        action = action[:,0,None].expand(batch_size,num_buckets)
        action=action[:,None]
        with torch.no_grad():
            gammas_one=torch.ones(batch_size,n,1,dtype=torch.float,device='cuda')
            gamma_step = 1-torch.tensor(( (schedule_max_step - min(grad_step, schedule_max_step)) / schedule_max_step) * (initial_gamma-final_gamma) + final_gamma).exp()
            gammas=gammas_one*gamma_step

            
            returns = []
            for t in range(n):
                ret = 0
                for u in reversed(range(t, n)):
                    ret += torch.prod(c_flag[:,t+1:u+1],-2)*torch.prod(gammas[:,t:u],-2)*rewards[:,u+1]
                returns.append(ret)
            returns = torch.stack(returns,1)
        
        plot_vs = returns.clone().sum(-1)
        
        same_traj = (torch.prod(c_flag[:,:n],-2)).squeeze()
        
        returns = returns[:,0]
        returns = returns + torch.prod(gammas[0,:10],-2).squeeze()*same_traj[:,None]*model.support[None,:]
        returns = returns.squeeze()
        
        next_values = next_values[:,0]

        log_probs = torch.log(q[:,0].gather(-2, action)[:,None] + eps).contiguous()
        
        
        dist = project_distribution(returns, next_values.squeeze(), model.support)
        
        loss = -(dist*(log_probs.squeeze())).sum(-1).view(batch_size,-1).sum(-1)
        dqn_loss = loss.clone().mean()
        td_error = (loss + torch.nan_to_num((dist*torch.log(dist))).sum(-1)).mean()

        
        batched_loss = loss.clone()
        
        
        z_tgt = F.normalize(z[:,1:], 2, dim=-1, eps=1e-5)
        z_pred = F.normalize(z_pred, 2, dim=-1, eps=1e-5)

        
        recon_loss = (mse(z_pred.contiguous().view(-1,HIDDENS), z_tgt.contiguous().view(-1,HIDDENS))).sum(-1)
        recon_loss = 5*(recon_loss.view(batch_size, -1).mean(-1))*same_traj
        
        
        loss += recon_loss
        
        loss = (loss*is_w).mean() # mean across batch axis

    loss.backward()

    param_norm, grad_norm = params_and_grad_norm(model)
    #scaler.scale(loss).backward()
    #scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
    #scaler.step(optimizer)
    #scaler.update()
    
    optimizer.step()
    optimizer.zero_grad()
    
    #memory.set_priority(idxs, batched_loss)
    memory.set_priority(idxs, batched_loss, same_traj)
    

    lr = optimizer.param_groups[0]['lr']
    
    logs = {'loss': loss, 'dqn_loss': dqn_loss, 'recon_loss': recon_loss.mean(), 'lr': lr, 'returns': plot_vs.mean(),
               'buffer rewards': rewards.mean(0).sum(), 'is_w': is_w.mean(),
               'gamma': gamma_step, 'td_error': td_error, 'param_norm': param_norm.sum(), 'grad_norm': grad_norm.sum()}

    z = z[:,:5]
    diffusion_actions = F.one_hot(diffusion_actions[:,:5], n_actions)
    latent = torch.cat((z, diffusion_actions), -1)

    if grad_step>1500:
        logs = train_diffusion(latent, z[:,0], same_traj, std, is_w, logs)
    if grad_step>3000:
        logs = train_energy(latent, z[:,0], same_traj, std, is_w, logs)

    
    wandb.log(logs)




scores=[]
memory.free()
step=0
#model.share_memory()
grad_step=0

In [14]:
import torch
import math

m=8
n_sim=16
m=torch.tensor(m)

j = math.floor(n_sim/(torch.log2(m)*(m)))

j

0

In [33]:
reset_every=40000
idx = torch.tensor([24000, 4000, 35000, 20000, 16000, 15000, 41000, 55000, 56000, 9000, 96000])

steps = idx - reset_every*(idx//reset_every)
mask = (steps<reset_every*0.4).float() + (steps>reset_every*0.8).float()
mask

tensor([0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0.])

In [37]:
import torch

a=torch.tensor([[0.0945, 0.4497, 0.1342, 0.0141, 0.2003, 0.0612, 0.0460],
        [0.0865, 0.5353, 0.1180, 0.0130, 0.1491, 0.0535, 0.0445],
        [0.1096, 0.4248, 0.1380, 0.0168, 0.1932, 0.0678, 0.0498]],
       device='cuda:0')

a.argmax(-1)

tensor([1, 1, 1], device='cuda:0')

In [8]:
step=0


progress_bar = tqdm.tqdm(total=total_steps)

while step<(10):
    state, info = env.reset()
    state = preprocess(state)

    states = deque(maxlen=4)
    for i in range(4):
        states.append(state)
    
    actions_to_step = []
    
    eps_reward=torch.tensor([0], dtype=torch.float)
    
    reward=np.array([0])
    done_flag=np.array([False])
    terminated=np.array([False])

    last_lives=np.array([0])
    life_loss=np.array([0])
    resetted=np.array([0])

    last_grad_update=0
    while step<(total_steps):
        progress_bar.update(1)
        model_target.train()
        
        len_memory = len(memory)
        
        #if resetted[0]>0:
        #    states = env.noop_steps(states)
            
        
        
        if len(actions_to_step)==0:
            actions_to_step = epsilon_greedy(torch.cat(list(states),-3).unsqueeze(0), len_memory, grad_step, actions_to_step)
        action = actions_to_step.pop(0).squeeze()
        
        memory.push(torch.cat(list(states),-3).detach().cpu(), torch.tensor(reward,dtype=torch.float), action,
                    torch.tensor(np.logical_or(done_flag, life_loss),dtype=torch.bool))
        #print('action', action, action.shape)
        
        state, reward, terminated, truncated, info = env.step([action.numpy()])
        state = preprocess(state)
        states.append(state)
        
        eps_reward+=reward
        reward = reward.clip(-1, 1)


        
        done_flag = np.logical_or(terminated, truncated)
        lives = info['lives']
        life_loss = (last_lives-lives).clip(min=0)
        resetted = (lives-last_lives).clip(min=0)
        last_lives = lives

        
        n = int(initial_n * (final_n/initial_n)**(min(grad_step,schedule_max_step) / schedule_max_step))
        n = np.array(n).item()
        
        memory.priority[len_memory] = memory.max_priority()
        

        if len_memory>2000:
            for i in range(2):
                optimize(step, grad_step, n)
                target_model_ema(model, model_target)
                grad_step+=1

        
        if ((step+1)%10000)==0:
            save_checkpoint(model, model_target, optimizer, step,
                            'checkpoints/atari_last.pth')
        
            
        
        if grad_step>reset_every:
            #eval()
            print('Reseting on step', step, grad_step)
            
            #seed_np_torch(random.randint(SEED-1000, SEED+1000)+step)
            random_model = DQN(n_actions).cuda()
            model.hard_reset(random_model)
            
            #seed_np_torch(random.randint(SEED-1000, SEED+1000)+step)
            random_model = DQN(n_actions).cuda()
            model_target.hard_reset(random_model)
            
            random_model = Diffusion(n_actions).cuda()
            diffusion_model.hard_reset(random_model)

            random_model = EnergyModel(n_actions).cuda()
            energy_model.hard_reset(random_model)
            
            seed_np_torch(SEED)
            
            random_model=None
            grad_step=0

            actor_modules=[model.prediction, model.projection, model.a, model.v]
            params_ac=[]
            for module in actor_modules:
                for param in module.parameters():
                    params_ac.append(param)
                    

            perception_modules=[model.encoder_cnn, model.transition]
            params_wm=[]
            for module in perception_modules:
                for param in module.parameters():
                    params_wm.append(param)
            
            optimizer_aux = torch.optim.AdamW(params_wm, lr=lr, weight_decay=0.1, eps=1.5e-4)
            copy_states(optimizer, optimizer_aux)
            optimizer = torch.optim.AdamW(chain(params_wm, params_ac),
                                lr=lr, weight_decay=0.1, eps=1.5e-4)
            copy_states(optimizer_aux, optimizer)

        
            optimizer_energy = torch.optim.AdamW(energy_model.parameters(),
                                lr=lr_energy, weight_decay=0.)
        
        
        
        step+=1
        
        log_t = done_flag.astype(float).nonzero()[0]
        
        if len(log_t)>0:
            for log in log_t:
                wandb.log({'eps_reward': eps_reward[log].sum()})
                scores.append(eps_reward[log].clone())
            eps_reward[log_t]=0

save_checkpoint(model, model_target, optimizer, step, f'checkpoints/{env_name}_diff.pth')


  2%|▏         | 1971/102000 [00:05<04:06, 405.57it/s]C:\Users\sneep\AppData\Local\Temp\ipykernel_13612\700723638.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gamma_step = 1-torch.tensor(( (schedule_max_step - min(grad_step, schedule_max_step)) / schedule_max_step) * (initial_gamma-final_gamma) + final_gamma).exp()
  3%|▎         | 2752/102000 [02:48<5:38:05,  4.89it/s]C:\Users\sneep\anaconda3\envs\python_\Lib\site-packages\nosaveddata\builders\transformer.py:66: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=is_causal)
  3%|▎         | 3503/102000 [07:00<17:02:38,  1.61it/s]

  3%|▎         | 3504/102000 [07:03<31:32:36,  1.15s/it]

  3%|▎         | 3505/102000 [07:05<41:54:56,  1.53s/it]

  3%|▎         | 3506/102000 [07:07<49:12:55,  1.80s/it]

  3%|▎         | 3507/102000 [07:10<54:23:03,  1.99s/it]

...

  4%|▍         | 4500/102000 [47:20<63:27:19,  2.34s/it]

  4%|▍         | 4501/102000 [47:23<63:34:04,  2.35s/it]

  4%|▍         | 4502/102000 [47:25<64:04:03,  2.37s/it]

  4%|▍         | 4503/102000 [47:27<64:10:52,  2.37s/it]


KeyboardInterrupt



In [ ]:
load_to_eval=False

if load_to_eval:
    model.load_state_dict(torch.load(f'checkpoints/{env_name}_diff.pth')['model_state_dict'])
    model_target.load_state_dict(torch.load(f'checkpoints/{env_name}_diff.pth')['model_target_state_dict'])

In [ ]:
import matplotlib.pyplot as plt

num_envs=1

#env = gym.vector.make(f"{env_name}NoFrameskip-v4", num_envs=num_envs, render_mode='human')
env = gym.vector.make(f"{env_name}NoFrameskip-v4", num_envs=num_envs)
env = MaxLast2FrameSkipWrapper(env,seed=SEED)

def eval_phase(eval_runs=50, max_eval_steps=27000, num_envs=1):
    progress_bar = tqdm.tqdm(total=eval_runs)
    
    scores=[]
    
    state, info = env.reset()
    state = preprocess(state)
    print(f"init state {state.shape}")
    
    states = deque(maxlen=4)
    for i in range(4):
        states.append(state)

    actions_to_step = []
    
    eps_reward=torch.tensor([0]*num_envs, dtype=torch.float)
    
    reward=np.array([0]*num_envs)
    terminated=np.array([False]*num_envs)
    
    last_lives=np.array([0]*num_envs)
    life_loss=np.array([0]*num_envs)
    resetted=np.array([0])

    finished_envs=np.array([False]*num_envs)
    done_flag=0
    last_grad_update=0
    eval_run=0
    step=np.array([0]*num_envs)
    while eval_run<eval_runs:
        #seed_np_torch(SEED+eval_run)
        env.seed=SEED+eval_run
        model_target.train()
        
        #if resetted[0]>0:
        #    states = env.noop_steps(states)
        
        if len(actions_to_step)==0:
            actions_to_step = epsilon_greedy(torch.cat(list(states),-3).unsqueeze(0), len_memory, grad_step=35000, actions_to_step=actions_to_step).cpu()
        action = actions_to_step.pop(0).squeeze()
        
        state, reward, terminated, truncated, info = env.step([action.numpy()] if num_envs==1 else action.numpy())
        state = preprocess(state)
        states.append(state)
        
        eps_reward+=reward

        
        done_flag = np.logical_or(terminated, truncated)
        lives = info['lives']
        life_loss = (last_lives-lives).clip(min=0)
        resetted = (lives-last_lives).clip(min=0)
        last_lives = lives        
        
        step+=1
        
        log_t = done_flag.astype(float).nonzero()[0]
        if len(log_t)>0:# or (step>max_eval_steps).any():
            progress_bar.update(1)
            for log in log_t:
                #wandb.log({'eval_eps_reward': eps_reward[log].sum()})
                if finished_envs[log]==False:
                    scores.append(eps_reward[log].clone())
                    eval_run+=1
                    #finished_envs[log]=True
                step[log]=0
                
            eps_reward[log_t]=0            
            for i, log in enumerate(step>max_eval_steps):
                if log==True and finished_envs[i]==False:
                    scores.append(eps_reward[i].clone())
                    step[i]=0
                    eval_run+=1
                    eps_reward[i]=0
                    #finished_envs[i]=True
            
    return scores



def eval(eval_runs=50, max_eval_steps=27000, num_envs=1):
    assert num_envs==1, 'The code for num eval envs > 1 is messed up.'
    
    scores = eval_phase(eval_runs, max_eval_steps, num_envs)    
    scores = torch.stack(scores)
    scores, _ = scores.sort()
    
    _25th = eval_runs//4

    iq = scores[_25th:-_25th]
    iqm = iq.mean()
    iqs = iq.std()

    print(f"Scores Mean {scores.mean()}")
    print(f"Inter Quantile Mean {iqm}")
    print(f"Inter Quantile STD {iqs}")

    
    plt.xlabel('Episode (Sorted by Reward)')
    plt.ylabel('Reward')
    plt.plot(scores)
    
    new_row = {'env_name': env_name, 'mean': scores.mean().item(), 'iqm': iqm.item(), 'std': iqs.item(), 'seed': SEED}
    add_to_csv('results.csv', new_row)

    with open(f'results/{env_name}-{SEED}.txt', 'w') as f:
        f.write(f" Scores Mean {scores.mean()}\n Inter Quantile Mean {iqm}\n Inter Quantile STD {iqs}")
    
    
    return scores

scores = eval(eval_runs=100, num_envs=1)

In [ ]:
'''
import pandas as pd
new_row = {'env_name': "Amidar", 'mean': 11.0, 'iqm': 11.0, 'std': 11.0, 'seed': 000}

df = pd.read_csv('results.csv',sep=',')
df.loc[len(df.index)] = new_row    
#df.to_csv('results.csv', index=False)

df
'''
# Add to csv suddenly stopped working